# Sample LASSO task functions

In [ ]:
import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


# if os.path.exists(os.path.abspath('../src')):
#     sys.path.append(os.path.abspath('../src'))

from src.feat_build import main
from src.model_build import train

In [ ]:
inv_dir = Path(os.getcwd())
proj_dir = inv_dir.parent

sample_guids_parquet = 'sample_guid_10000_china_us.parquet'
directories = ["frgnd_backgrnd_apps_v4_hist", "web_cat_usage_v2","power_acdc_usage_v4_hist","os_c_state", "hw_pack_run_avg_pwr"]

if 'feat.parquet' not in os.listdir(inv_dir / 'out'):
    main.generate_features(sample_guids_parquet, inv_dir, directories)
else:
    print('Features already generated')

In [ ]:
# Generates Synthetic Data
# main.generate_synthetic_data(proj_dir / 'dummy_data')
# syn_feat = pd.read_parquet(proj_dir / "dummy_data" / "synthetic_data.parquet")
# syn_feat.shape

Read in Data

In [ ]:
feat = pd.read_parquet(os.path.join('out', 'feat.parquet'))
feat.shape

Run baseline non-private case with no regularization:

In [ ]:
test_mse, feat_dict, r2 = train.train(feat, "lasso", tol=1e-4, l=0)
test_mse, r2

Compare multiple methods with varying regularization: $l$

In [ ]:
# compare methods
results = pd.DataFrame(columns=["coord_desc", "fw_lap", "fw_exp"])
for l in [0.25, 0.5, .9, 1, 5, 10, 25, 100]:
    # print("parameter: ", l)
    test_mse1, feat_dict, r2 = train.train(feat, "lasso", tol=1e-4, l=1/l) 
    test_mse2, feat_dict, r2 = train.train(feat, "fw-lasso-lap", tol=1e-4, l=l, max_iter=2500)
    test_mse3, feat_dict, r2 = train.train(feat, "fw-lasso-exp", tol=1e-4, l=l, max_iter=2500)
    results.loc[l] = [test_mse1, test_mse2, test_mse3]

In [ ]:
results

In [ ]:
results.plot(kind='line', logx=True);

Test many epsilon values at regularization $l=10$

In [ ]:
epss = [0.25, 0.5, 1, 5, 10, 100, 10_000]
epsresults = pd.DataFrame(columns=["fw_lap", "fw_exp"])
for eps in epss:
    print("parameter: ", eps)
    test_mse1, feat_dict, r2 = train.train(feat, "fw-lasso-lap", tol=1e-4, l=10, epsilon=eps, max_iter=2500)
    test_mse2, feat_dict, r2 = train.train(feat, "fw-lasso-exp", tol=1e-4, l=10, epsilon=eps, max_iter=2500)
    epsresults.loc[eps] = [test_mse1, test_mse2]

In [ ]:
epsresults

In [ ]:
epsresults.plot(kind='line', logx=True);

Example of using sigmoid utility mapping on exponential implementation:

In [ ]:
l = 10
tol = 1e-4
max_iter = 2500
epss = [0.01, 0.1, 1, 10, 100]
epsresults = []

for eps in epss:
    test_mse, feat_dict, r2 = train.train(feat, "fw-lasso-exp", tol=tol, l=l, epsilon=eps, max_iter=max_iter)
    
    epsresults.append(test_mse)

rmses = np.sqrt(np.array(epsresults))
max_rmse = np.max(rmses)
c = .1 # higher values punish rmse more
utility = 2 / (1 + np.exp(c * rmses / max_rmse))

In [ ]:
print(dict().get("key"))

In [ ]:
plt.plot(epss, utility)
plt.xlabel('epsilon (log scaled)')
plt.ylabel('utility')
plt.ylim(0, 1)
plt.xscale('log')